In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [4]:
#Read in csv files

napaData_df = pd.read_csv('resources/napa_weather.csv') # replcae with correct file path
marinData_df = pd.read_csv('resources/marin_weather.csv') # replcae with correct file path
placerData_df = pd.read_csv('resources/placer_weather.csv') # replcae with correct file path
santaCruzData_df = pd.read_csv('resources/santa_cruz_weather.csv') # replcae with correct file path
sonomaData_df = pd.read_csv('resources/santa_cruz_weather.csv') # replcae with correct file path
caliWine_df = pd.read_csv('resources/Californa_Wine_Production_1980_2020.csv')  # replcae with correct file path

In [16]:
# Clean caliWine df to only include top five price per ton counties 

#reduce dataframe to only include relevant columns
cleaned_wine_df = caliWine_df[['Year', 'County',
       'HarvestedAcres', 'Yield(Unit/Acre)', 'Production',
       'Price(Dollars/Unit)', 'Unit', 'Value(Dollars)']].copy()

cleaned_wine_df.head()




,Year,County,HarvestedAcres,Yield(Unit/Acre),Production,Price(Dollars/Unit),Unit,Value(Dollars)
0,2020,Alameda,2530.0,5.14,13000.0,1497.69,Tons,19470000
1,2020,Amador,5360.0,2.31,12400.0,1318.31,Tons,16347000
2,2020,Calaveras,579.0,3.06,1770.0,1325.99,Tons,2347000
3,2020,Colusa,747.0,6.02,4500.0,684.67,Tons,3081000
4,2020,ContraCosta,1940.0,4.69,9090.0,751.27,Tons,6829000


In [ ]:
#drop all rows that do not include top five price per ton counties

In [9]:
#Merge all weather data into one dataframe and split year

#insert column for county name
marinData_df.insert(1, 'Name', 'Marin')
napaData_df.insert(1, 'Name', 'Napa')
placerData_df.insert(1, 'Name', 'Placer')
santaCruzData_df.insert(1, 'Name', 'SantaCruz')
sonomaData_df.insert(1, 'Name', 'Sonoma')

#merge all dataframes

merged_weather_df = pd.concat([marinData_df, napaData_df, placerData_df, santaCruzData_df, sonomaData_df], axis = 0)

# split the month column into months and years column
merged_weather_df[['month', 'year']] = merged_weather_df['Month'].str.split(',', expand=True)


merged_weather_df.head()

ValueError: cannot insert Name, already exists

In [12]:
#drop 'Month' column and save new dataframe

merged_weather_df = merged_weather_df.drop('Month', axis=1)

,Name,Avg. Temp,Min. Temp,Max. Temp,Precip.,month,year
0,Marin,51.5,43.2,59.7,2.83,December,2020
1,Marin,53.5,44.2,62.8,1.54,November,2020
2,Marin,66.2,54.4,78.0,0.01,October,2020
3,Marin,66.8,56.7,77.0,0.00,September,2020
4,Marin,66.7,56.4,77.1,0.25,August,2020
...,...,...,...,...,...,...,...
487,Sonoma,55.0,44.6,65.4,0.58,May,1980
488,Sonoma,53.9,42.7,65.2,2.62,April,1980
489,Sonoma,50.9,40.1,61.7,2.67,March,1980
490,Sonoma,52.6,44.2,61.0,11.91,February,1980
